# Module 6 - Aggregation Examples

In [ ]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None

### Group By

In [ ]:
combined = pd.read_csv('Combined_airbnb_listings.csv') #read in combined dataframe
combined

By simply running a groupby, we will get a GroupBy object, but do not actually see data

In [ ]:
combined.groupby('location')

To see data, we typically apply a function to the groupby. Let's try to take a look at how price differs by location

In [ ]:
combined.groupby('location')['price'].mean()

Hmm, it looks like our aggregation failed. This can happen if there are non-numeric data points in your dataset.  Let's take a look and fix this.

In [ ]:
combined['price'].head() #we can see there's a problem becuase there is a dollar sign included, which makes it non-numeric
combined['price'] = pd.to_numeric(combined['price'].replace('[\$,]', '', regex=True))
combined['price']

#while it won't skew our results, we may also want to note that there are some NaN values in the price column. This doesn't matter for groupby (it will just ignore these rows when aggregating), but this may be a sign of bad data that we don't want to include in the overall dataset
combined['price'].isna().sum() #counts the number of NaN values in the column

While a more sophisticated method would be to interpolate a price based on nearby properties as well as the size/attributes/amenities of the property, for the sake of not getting off track on the topic we will just go ahead and remove these datapoints (it wouldn't make sense to have a price of 0 or something else as that would skep the average)

While we're at it, we will also go ahead and clean up the dataset a bit with some of the methods we have previously used.  In the real world, always verify that your assumptions about one dataframe are still valid before performing applying it to a full combined dataset!


In [ ]:
cols_to_delete = ['neighbourhood_group_cleansed','calendar_updated','license','scrape_id','last_scraped','source','listing_url','picture_url','host_url','host_thumbnail_url','host_picture_url']

combined = combined.dropna(subset=['price']).drop(cols_to_delete,axis=1).drop_duplicates()[(combined['property_type'].str.contains('hotel')==False) & (combined['room_type'].str.contains('Hotel')==False)]
combined.groupby('location')['price'].mean()

That seems like a large difference between the cities...let's dig deeper.

In [ ]:
combined.groupby('location')['price'].describe()

Istanbul prices seem to be significantly higher across the board. However, if we look back at the data dictionary, we will note that prices are in "local currency". Therefore, to be able to truly compare, we really need to convert Turkish Lira to USD. This is clearly a bit of a tangent from the goal of this notebook, but demonstrates how things often work in the real-world -- to solve a problem, you often need to solve a few other problems first. This library we are using to do this is not a particularly common library to use, but demonstrates that sometimes you will need to look for and use niche libraries to accomplish a task.

In [ ]:
from currency_converter import ECB_URL, CurrencyConverter #remember to install this first with `pip install CurrencyConverter`
from datetime import date
import urllib #allows us to 
import os

filename = f"ecb_{date.today():%Y%m%d}.zip" #specifies the name of the target file
if not os.path.isfile(filename): #checks if the file already exists
    urllib.request.urlretrieve(ECB_URL, filename) #if not, it will query the URL defined in the library and pull the data, which can be used to convert currency
c = CurrencyConverter(filename) #creates a CurrencyConverter object from the data

In [ ]:
c.convert(100,'USD','TRY') #simple example showing how the conversion of 100 USD to TRY

#Now, let's do this for the full dataframe
combined['price'] = combined.apply(lambda x: round(c.convert(x['price'], 'TRY','USD'),2) if x['location']=='Istanbul' else x['price'], axis=1) #this is an example of using conditionals and calling another function inside a lambda function.  Essentially, we are saying that we want to convert the currency if the location is istanbul, and if not, keep it with the current value
combined.groupby('location')['price'].describe()

Now, let's look at this with multi-indexing.  Clearly there is a difference between cities, but what about the difference in terms of both cities and type of accomodation?

In [ ]:
combined.groupby(['location','room_type'])['price'].agg(['size','mean']) # adding multiple columns to the groupby allows us to look at more specific subgroups, 

This certainly shows a different picture than before! In the real world, we'd want to continue an EDA by digging deeper into the possible layers and subgroups to fully understand the connection between the different variables and similarities/differences between the different groups, but for the sake of staying on topic with these examples, we will move on.

Finally, remember that the objects returned by groupby statements are GroupBy objects, not dataframes.  If you wanted to work with this like a dataframe to do any further analysis (sort, filter, etc.), you would need to convert this back to a dataframe, which you can do by resetting the index.

In [ ]:
grouped = combined.groupby(['location','room_type'])['price'].agg(['size','mean']).reset_index() # resetting the index flattens the groupby so that every row has a value for every column
type(grouped)
grouped

#Now, we can sort, filter, etc.
grouped.sort_values(by='mean',ascending=False)

### Pivot Tables
As discussed, pivot tables and group by objects are quite related and the source code for a pivot table actually uses groupby! How you use pivots versus groupbys is partly personal preference. Let's try to do the same aggregations with a pivot table as we did with groupby and see how they look different.

In [ ]:
pd.pivot_table(combined,values='price',index='location') 

pd.pivot_table(combined,values='price',index=['location','room_type'],aggfunc=('size','mean'))

#These look the same as our groupby results above!

While you can always get the same results, pivot tables can sometimes help display data in a certain way easier

In [ ]:
room_prices = pd.pivot_table(combined,values='price',index=['location','room_type'],columns='property_type',margins=True) #This looks at the average price by location, room type, and property type. Margins in this case are a weighted average by grouping
room_prices

### Stacking and Unstacking

In [ ]:
# To view data like you would with a groupby from a pivot, we can simply stack the results, although we will need to change the output to be able to see all of the results
pd.options.display.max_rows = 300
room_prices.stack()